In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
#!pip install ml-dtypes==0.5.0
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.18.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-06-27 00:32:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2025-06-27 00:32:14 (7.60 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-06-27 00:32:14--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
df_train = pd.read_csv(filepath_or_buffer=train_file_path,sep='\t',header=None,names=['label','sms'])
df_train.head()

,label,sms
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [4]:
df_test = pd.read_csv(filepath_or_buffer=test_file_path,sep='\t',header=None,names=['label','sms'])
df_test.head()

,label,sms
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [5]:
df_train['label'] = pd.Categorical(df_train['label']).codes
df_train['label'] = df_train['label'].astype(np.int32) # Explicitly convert to int32
df_train.head()

,label,sms
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [6]:
df_test['label'] = pd.Categorical(df_test['label']).codes
df_test.head()

,label,sms
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [7]:
VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(df_train['sms'])

In [8]:
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=64,mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',keras.metrics.binary_accuracy])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(encoder(df_train['sms']), df_train['label'], batch_size=32, epochs=10, verbose=1, validation_split=0.3)

Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 25s 276ms/step - acc: 0.9998 - binary_accuracy: 0.9998 - loss: 0.0014 - val_acc: 0.9809 - val_binary_accuracy: 0.9809 - val_loss: 0.1167
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 39s 252ms/step - acc: 0.9999 - binary_accuracy: 0.9999 - loss: 0.0013 - val_acc: 0.9809 - val_binary_accuracy: 0.9809 - val_loss: 0.1213
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 253ms/step - acc: 0.9993 - binary_accuracy: 0.9993 - loss: 0.0029 - val_acc: 0.9793 - val_binary_accuracy: 0.9793 - val_loss: 0.1302
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 24s 260ms/step - acc: 0.9997 - binary_accuracy: 0.9997 - loss: 0.0016 - val_acc: 0.9793 - val_binary_accuracy: 0.9793 - val_loss: 0.1331
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 256ms/step - acc: 0.9999 - binary_accuracy: 0.9999 - loss: 7.9997e-04 - val_acc: 0.9817 - val_binary_accuracy: 0.9817 - val_loss: 0.1353
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 246ms/step - acc: 0.9983 - binary_accuracy: 0.9983 - loss: 0.0051 - val_acc: 0.

In [14]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  inputstr = encoder(pred_text)
  inputstr = tf.expand_dims(inputstr, 0)
  pred = model.predict(inputstr)[0][0]

  if pred >= 0.5:
    label = "spam"
  else:
    label = "ham"
  return [pred, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[np.float32(2.91241e-08), 'ham']


In [15]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
You passed the challenge. Great job!
